<a href="https://www.kaggle.com/code/rputtam/pyspark-explore?scriptVersionId=188030392" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## This notebook aims to utilize pyspark and explore various functions for data analysis and manipulation.

Resources for pySpark 
1. https://sparkbyexamples.com/pyspark-tutorial/
2. https://github.com/spark-examples/pyspark-examples
3. https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html?hl=en-GB

##### There are two types of pyspark rdd operations - transformations and actions
* Transformations return rdds
* Actions return the result (non-rdds)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Install PySpark

In [1]:
# Installing Pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=14ab0f7b0e178968bcef2917df0c814686f84ec82553e007e9e1afbeb25b495b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


## Create SparkSession

In [2]:
# Importing and creating SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('explore_pyspark').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/12 22:42:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
# Importing necessary packages
from pyspark.sql.types import *
#from pyspark.sql.functions import *
import pyspark.sql.functions as F

## Loading data

In [38]:
# Reading the kaggle dataset without inferschema
df = spark.read.csv('/kaggle/input/employees-performance-for-hr-analytics/Uncleaned_employees_final_dataset (1).csv',header=True)
df.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)
 |-- education: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- recruitment_channel: string (nullable = true)
 |-- no_of_trainings: string (nullable = true)
 |-- age: string (nullable = true)
 |-- previous_year_rating: string (nullable = true)
 |-- length_of_service: string (nullable = true)
 |-- KPIs_met_more_than_80: string (nullable = true)
 |-- awards_won: string (nullable = true)
 |-- avg_training_score: string (nullable = true)



In [39]:
# Reading the kaggle dataset with inferschema and checking the data types
df = spark.read.csv('/kaggle/input/employees-performance-for-hr-analytics/Uncleaned_employees_final_dataset (1).csv',inferSchema= True,header=True)
#inferSchema infers the datatype of field from its values. But it reads the entire data and may have performance issues while reading larger datasets.
#The otherway is to predefine the schema using structType that saves time - Optimized approach
df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)
 |-- education: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- recruitment_channel: string (nullable = true)
 |-- no_of_trainings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- previous_year_rating: integer (nullable = true)
 |-- length_of_service: integer (nullable = true)
 |-- KPIs_met_more_than_80: integer (nullable = true)
 |-- awards_won: integer (nullable = true)
 |-- avg_training_score: integer (nullable = true)



In [ ]:
#Pre-defining the schmea
# Note, all the fields available in the data need to be defined, the size of the structType and dataframe should match

sch = StructType([
    StructField("employee_id",IntegerType(),True),
    StructField("department",StringType(),True),
    StructField("region",StringType(),True),
    StructField("education",StringType(),True),
    StructField("gender",StringType(),True),
    StructField("recruitment_channel",StringType(),True),
    StructField("no_of_trainings",IntegerType(),True),
    StructField("age",IntegerType(),True),
    StructField("previous_year_rating",IntegerType(),True),
    StructField("length_of_service",IntegerType(),True),
    StructField("KPIs_met_more_than_80",IntegerType(),True),
    StructField("awards_won",IntegerType(),True),
    StructField("avg_training_score",IntegerType(),True)
])

In [ ]:
df3 = spark.read\
.option("header",True) \
.schema(sch) \
.csv('/kaggle/input/employees-performance-for-hr-analytics/Uncleaned_employees_final_dataset (1).csv')

df3.printSchema()

## withColumn()

### .lit()

In [ ]:
# Creating new column with constant value
df2 = df.withColumn("company",F.lit('ABC'))\
        .withColumn("country", F.lit('USA'))
df2.select('employee_id','company','country').show(2)

In [ ]:
# Changing column datatypes of selected fields using withColumn

df2 = df.withColumn("age",df.age.cast(IntegerType())) \
.withColumn("length_of_service",df.length_of_service.cast(IntegerType()))
df2.printSchema()

In [ ]:
# Rename column
df2 = df2.withColumnsRenamed({'country':'nation','company':'org'})
df2.printSchema()

In [ ]:
df2=df2.drop("org","nation")
df2.columns

## .select()

In [ ]:
# selecting few fields using select
x = df3.select('employee_id','age')
print(type(x))
x.show(5)

## .filter()

In [ ]:
#filter with one condition
df3.filter(df3.age > 20).select('employee_id','age').show(5)

In [ ]:
#Filter with multiple conditions
df3.filter( (df3.age >20) & (df3.department == "HR") ).select('employee_id',
                                                              'department',
                                                              'age').show(4)

In [ ]:
# Finance and age>40 or HR 
df4 = df3.filter( ( (df3.department == "Finance") & (df3.age>40) ) | (df3.department == "HR") )\
        .select('employee_id','department','age')

In [ ]:
df4.groupBy('department')\
.count()\
.show()

In [ ]:
df4.filter((df4.department=="Finance") & (df4.age<40)).show()

### Get unique values for categorical fields

In [ ]:
df3.select('department')\
.distinct().show()

In [ ]:
# to get unique values as a list
df3.agg(F.collect_set("department")).collect()[0][0]

In [ ]:
df3.agg(F.collect_set("recruitment_channel")).collect()[0][0]

In [ ]:
dept =['R&D','Technology','Analytics']
df4=df3.filter(df3.department.isin(dept))
df4.select(df4.department).distinct().show()

In [ ]:
df3.columns

## Aggregate functions

In [ ]:
df3.groupBy(['department'])\
   .agg(F.round(F.median('age'),2).alias('median_age'),
       F.mean('avg_training_score').alias('avg_training_score'),
        F.mode('length_of_service').alias('max_los')
       ).show()

In [ ]:
df3.groupBy(['recruitment_channel'])\
   .agg(F.median('age').alias('median_age'),
       F.mean('no_of_trainings').alias('trainings_avg'))\
   .show()

## .explode()

In [ ]:
#Let's create a new column-languages known for each employee
# For simplicity, creating same values for all employees

df3=df3.withColumn('languages',F.lit(['SQL','Python']))
df3.select('employee_id','languages').show(5)

In [ ]:
df_lang = df3.select(df3.employee_id,F.explode(df3.languages))
df_lang.show(5)

##### explode can also be used to breakdown a column in dictionary format similar to list. This creates two new columns - key and value

## udf function

##### UDF is User Defined Function that can be used to modify columns or fields

In [ ]:
#udf on one column

# Define function
def cap_gender(g):
    return g.upper()

# udf creation
gen_udf = F.udf(cap_gender,StringType())

# Apply udf
df_gen = df3.withColumn('gender_upd',gen_udf(df3.gender))

In [ ]:
df_gen.select('gender','gender_upd').show(3)

##### UDF with multiple returntypes

In [ ]:
def modify_dataframe(department,recruitment_channel):
    mod_dept = department.upper()
    mod_rec_channel = recruitment_channel.title()
    return mod_dept,mod_rec_channel

In [ ]:
#Register UDF
df_udf = F.udf(modify_dataframe,StructType([StructField('mod_dept',StringType()),
                                           StructField('mode_rec_channel',StringType())]))

In [ ]:
#Apply udf to dataframe
df4 = df3.withColumn('mod_columns',df_udf(df3['department'],df3['recruitment_channel']))

In [ ]:
df4.select('mod_columns').show(4)

In [ ]:
#Split the udf updated columns
df5=df4.select('mod_columns.*').drop('mod_columns')
df5.show(4)

### F.when()

Creating a column based on condition

In [55]:
df.dtypes

[('employee_id', 'int'),
 ('department', 'string'),
 ('region', 'string'),
 ('education', 'string'),
 ('gender', 'string'),
 ('recruitment_channel', 'string'),
 ('no_of_trainings', 'int'),
 ('age', 'int'),
 ('previous_year_rating', 'int'),
 ('length_of_service', 'int'),
 ('KPIs_met_more_than_80', 'int'),
 ('awards_won', 'int'),
 ('avg_training_score', 'int'),
 ('training_result', 'int')]

In [58]:
df = df.withColumn('avg_training_score',df['avg_training_score'].cast(DoubleType())) 

In [59]:
df.dtypes

[('employee_id', 'int'),
 ('department', 'string'),
 ('region', 'string'),
 ('education', 'string'),
 ('gender', 'string'),
 ('recruitment_channel', 'string'),
 ('no_of_trainings', 'int'),
 ('age', 'int'),
 ('previous_year_rating', 'int'),
 ('length_of_service', 'int'),
 ('KPIs_met_more_than_80', 'int'),
 ('awards_won', 'int'),
 ('avg_training_score', 'double'),
 ('training_result', 'int')]

In [60]:
df.createOrReplaceTempView("df_sql")

In [61]:
spark.sql("select min(avg_training_score),max(avg_training_score) from df_sql").show()

+-----------------------+-----------------------+
|min(avg_training_score)|max(avg_training_score)|
+-----------------------+-----------------------+
|                   39.0|                   99.0|
+-----------------------+-----------------------+



In [64]:
df = df.withColumn('training_result_fg', F.when(F.col('avg_training_score') > 50, 1).otherwise(0))

In [68]:
df.select(['avg_training_score','training_result_fg']).show(5)

+------------------+------------------+
|avg_training_score|training_result_fg|
+------------------+------------------+
|              77.0|                 1|
|              51.0|                 1|
|              47.0|                 0|
|              65.0|                 1|
|              61.0|                 1|
+------------------+------------------+
only showing top 5 rows

